In [64]:
import pandas as pd
import numpy as np
import unicodedata as ud

In [65]:
cotisantsPSL = pd.read_excel('cotisantsPSL.xlsx')
cotisantsPSL.head(2)

,Référence commande,Date de la commande,Statut de la commande,Nom adhérent,Prénom adhérent,Carte d'adhérent,Nom payeur,Prénom payeur,Email payeur,Raison sociale,Moyen de paiement,Tarif,Montant tarif,Code Promo,Montant code promo,Commentaires
0,101801293,2024-09-12 14:44:00,Validé,Kervin,Chauke,Voir la carte d'adhérent - PDF,Kervin,Chauke,chaukekervin@gmail.com,NaN,Carte bancaire,Cotisation PSL Sport,20,NaN,NaN,NaN
1,101772443,2024-09-12 11:30:00,Validé,ROBERT,Quentin,Voir la carte d'adhérent - PDF,ROBERT,Quentin,quentin.robert@etu.minesparis.psl.eu,NaN,Carte bancaire,Cotisation PSL Sport,20,NaN,NaN,NaN


In [66]:
def normalize_name(name):
    return name.capitalize()

def remove_accents(input_str):
    # Normalisation en forme 'NFKD' pour décomposer les accents
    nfkd_form = ud.normalize('NFKD', input_str)
    # Garder uniquement les caractères non diacritiques (sans accent)
    return ''.join([c for c in nfkd_form if not ud.combining(c)])

In [67]:
cotisantsPSL = pd.DataFrame({"Nom" : cotisantsPSL['Nom adhérent'], "Prénom" : cotisantsPSL['Prénom adhérent']})

cotisantsPSL['Nom'] = cotisantsPSL['Nom'].apply(normalize_name)
cotisantsPSL['Prénom'] = cotisantsPSL['Prénom'].apply(normalize_name)

cotisantsPSL['Nom'] = cotisantsPSL['Nom'].apply(remove_accents)
cotisantsPSL['Prénom'] = cotisantsPSL['Prénom'].apply(remove_accents)

cotisantsPSL['Nom Complet'] = cotisantsPSL['Nom'].str.strip() + ' ' + cotisantsPSL['Prénom'].str.strip()
list_cotisantsPSL = cotisantsPSL['Nom Complet'].tolist()
list_cotisantsPSL[:5]

['Kervin Chauke',
 'Robert Quentin',
 'Bourgarel Joseph',
 'Tannoury Charbel',
 'Picamal Fanny']

In [68]:
form_answer = pd.read_excel('formResults.xlsx')
form_answer.rename(columns={'Adresse mail ':'Adresse-mail'}, inplace=True)
form_answer.head(2)

,NOM,Prénom,Sexe,Genre,Adresse-mail,Numero de téléphone,Payé ?,Photo,Envoyé à PSL ?,Validé par PSL ?
0,SMAIL,Samy,M,Monsieur,samy.smail@etu.minesparis.psl.eu,661454702,OUI,23smail - Samy Smail.jpg,OUI,En attente
1,SORIN,Lucile,F,Madame,lucile.sorin@etu.minesparis.psl.eu,682250149,OUI,IMG_5050 - Lucile Sorin.jpeg,OUI,En attente


In [69]:
form_answer = pd.DataFrame({"Nom" : form_answer['NOM'], "Prénom" : form_answer['Prénom']})

form_answer['Nom'] = form_answer['Nom'].apply(normalize_name) #met une majuscule sur la première lettre puis tout en minuscule.
form_answer['Prénom'] = form_answer['Prénom'].apply(normalize_name)

form_answer['Nom'] = form_answer['Nom'].apply(remove_accents) #enlève les accents
form_answer['Prénom'] = form_answer['Prénom'].apply(remove_accents)

form_answer['Nom complet'] = form_answer['Nom'].str.strip() + ' ' + form_answer['Prénom'].str.strip()
list_form = form_answer['Nom complet'].tolist()
list_form[:5]

['Smail Samy',
 'Sorin Lucile',
 'Losantos Solene',
 'Bellicaud Alice',
 'Etheimer Paul']

In [70]:
list_prompt_payer = []
list_delinquent_payer = []
list_prompt_payer_without_form = []

list_prompt_payer = list(set(list_cotisantsPSL).intersection(set(list_form)))
#we got the prompt payer, now we want to see who answered the form but didn't pay us. Then, we would want to see who paid but didn't fill the form.

#those who answered the form without paying
list_delinquent_payer = list(set(list_form).difference(set(list_cotisantsPSL)))

#those who paid without filling the form
list_prompt_payer_without_form = list(set(list_cotisantsPSL).difference(set(list_form)))

#difference returns the elements that are present in the first object but not in the second.

print("Payeurs qui ont répondu au formulaire :", list_prompt_payer)
print("Répondants qui n'ont pas payé :", list_delinquent_payer)
print("Payeurs qui n'ont pas répondu au formulaire :", list_prompt_payer_without_form)
print("Nombre de payeurs qui ont répondu au formulaire :", len(list_prompt_payer))
print("Nombre de répondants qui n'ont pas payé :", len(list_delinquent_payer))
print("Nombre de payeurs qui n'ont pas répondu au formulaire :", len(list_prompt_payer_without_form))

Payeurs qui ont répondu au formulaire : ['Brunel Lenny', 'Smail Samy', 'Therond Barnabe', 'De ferran Louise', 'Tannoury Charbel', 'Sorin Lucile', 'Losantos Solene', 'Sterin Gaspard', 'Chapelant Antoine', 'Meddeb Paul', 'Robert Quentin', 'Djombi moussi Isabelle alexandra paola', 'Bikanda Lara maella', 'Royer Jules', 'Deteix Joseph', 'Durand Victor', 'Balzer Elke', 'Kervin Chauke', 'Guichaoua Gwenn', 'Lhayani Samuel', 'Picamal Fanny', 'Hucher Lenny', 'Dubois Camille', 'Etheimer Paul', 'Bourgarel Joseph', 'Cotty Clara', 'Breuil Paul', 'Kleinmann Hanna', 'Dupont Ludivine', 'Bellicaud Alice', 'Escudie Pierre']
Répondants qui n'ont pas payé : ['Zadora Martin', 'Baudry Nolan', 'Cormouls houles Blanche', 'Poux Raphael', 'Chabert Cyrianne', 'Andrieux Theo', 'Raspiller Franck', 'Piechaczyk Antonin', 'Zendron Hubert']
Payeurs qui n'ont pas répondu au formulaire : ['Hadj mohand Yanis', 'Vambert Tristan', 'De la rochefordiere Alfred']
Nombre de payeurs qui ont répondu au formulaire : 31
Nombre de r

In [72]:
list_prompt_payer = pd.DataFrame(list_prompt_payer, columns=['email'])
list_prompt_payer_without_form = pd.DataFrame(list_prompt_payer_without_form, columns=['email'])
list_delinquent_payer = pd.DataFrame(list_delinquent_payer, columns=['email'])

list_prompt_payer.to_json('data/prompt_payers.json', orient='records')
list_prompt_payer_without_form.to_json('data/prompt_payers_without_form.json', orient='records')
list_delinquent_payer.to_json('data/delinquent_payers.json', orient='records')